In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from google.colab import userdata
userdata.get('HF_TOKEN')
userdata.get('API_KEYS')

'f6ce85e7a4d211489452288775b28664a7e7883b'

In [15]:
import pandas as pd

file_path = "/content/drive/MyDrive/ABSA_Datasets/T-data.csv"
df = pd.read_csv(file_path)

# df = df[['News','Symbol','Aspect','Opinion','Sentiment']]
df.head()

,Date,Source,News,Symbol,Aspect,Opinion,Sentiment,Note
0,01/01/2024,https://thunhoon.com/article/285598,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,E-refune,หนุน,Positive,NaN
1,01/01/2024,https://thunhoon.com/article/285598,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,หุ้น,รับผลดี,Positive,NaN
2,01/01/2024,https://thunhoon.com/article/285598,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,Q1,โต,Positive,NaN
3,01/01/2024,https://thunhoon.com/article/285607,ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...,ILM,SSSG,เติบโตดี,Positive,NaN
4,02/01/2024,https://thunhoon.com/article/285638,IHLเป๋าตุงมกราทันที รับรู้รองเท้าแบรนด์ดัง,IHL,มกรา,เป๋าตุง,Positive,รายได้


In [16]:
df[df.isnull().any(axis=1)]

,Date,Source,News,Symbol,Aspect,Opinion,Sentiment,Note
0,01/01/2024,https://thunhoon.com/article/285598,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,E-refune,หนุน,Positive,NaN
1,01/01/2024,https://thunhoon.com/article/285598,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,หุ้น,รับผลดี,Positive,NaN
2,01/01/2024,https://thunhoon.com/article/285598,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,Q1,โต,Positive,NaN
3,01/01/2024,https://thunhoon.com/article/285607,ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...,ILM,SSSG,เติบโตดี,Positive,NaN
5,02/01/2024,https://thunhoon.com/article/285631,BLCรุกอาหารสุขภาพ อัพมูลค่าหนุนรายได้,BLC,รายได้,หนุน,Positive,NaN
...,...,...,...,...,...,...,...,...
794,12/02/2024,https://thunhoon.com/article/288195,หุ้น GLOBAL ราคาดีด หลังบอร์ดไฟเขียวจ่ายหุ้นปั...,GLOBAl,บอร์ด,ไฟเขียว,Positive,NaN
795,12/02/2024,https://thunhoon.com/article/288197,ILINKปักธงปี67พุ่ง10% ลุ้นผลประมูลงาน2พันล.,ILINK,NaN,พุ่ง,Positive,รายได้
796,12/02/2024,https://thunhoon.com/article/288197,ILINKปักธงปี67พุ่ง10% ลุ้นผลประมูลงาน2พันล.,ILINK,ผลประมูลงาน,ลุ้น,Neutral,NaN
797,12/02/2024,https://thunhoon.com/article/288199,TGE เผยบ.ย่อยชนะประมูลโรงไฟฟ้าขยะชุมชน ขนาด 9....,TGE,ประมูล,ชนะ,Positive,NaN


In [17]:
sent_tag = {
    'positive':'<POS>',
    'negative':'<NEG>',
    'neutral': '<NEU>',
}
SENTIMENT_TAGS = [v for k,v in sent_tag.items()] # ['<POS>','<NEG>','<NEU>']
SPECIAL_TAGS = ['<SYMBOL>','<ASPECT>','<OPINION>'] + SENTIMENT_TAGS

In [18]:
dataset_pre = pd.DataFrame(columns=['text','label'])
dataset_pre['text'] = df['News']

In [19]:
def to_label_text(label):
  label_text = ''
  if label.Symbol:
    label_text += f'<SYMBOL> {label.Symbol}'
  if label.Aspect:
    label_text += f'<ASPECT> {label.Aspect}'
  if label.Opinion:
    label_text += f'<OPINION> {label.Opinion}'
  if label.Sentiment:
    label_text += sent_tag[label.Sentiment.lower()]

  return label_text

In [20]:
dataset_pre['label'] = df.apply(lambda x: to_label_text(x), axis=1)
#df[df.isnull().any(axis=1)].apply(lambda x: to_label_text(x), axis=1) #

In [21]:
dataset_pre

,text,label
0,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO<ASPECT> E-refune<OPINION> หนุน<...
1,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO<ASPECT> หุ้น<OPINION> รับผลดี<POS>
2,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO<ASPECT> Q1<OPINION> โต<POS>
3,ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...,<SYMBOL> ILM<ASPECT> SSSG<OPINION> เติบโตดี<POS>
4,IHLเป๋าตุงมกราทันที รับรู้รองเท้าแบรนด์ดัง,<SYMBOL> IHL<ASPECT> มกรา<OPINION> เป๋าตุง<POS>
...,...,...
794,หุ้น GLOBAL ราคาดีด หลังบอร์ดไฟเขียวจ่ายหุ้นปั...,<SYMBOL> GLOBAl<ASPECT> บอร์ด<OPINION> ไฟเขียว...
795,ILINKปักธงปี67พุ่ง10% ลุ้นผลประมูลงาน2พันล.,<SYMBOL> ILINK<ASPECT> nan<OPINION> พุ่ง<POS>
796,ILINKปักธงปี67พุ่ง10% ลุ้นผลประมูลงาน2พันล.,<SYMBOL> ILINK<ASPECT> ผลประมูลงาน<OPINION> ลุ...
797,TGE เผยบ.ย่อยชนะประมูลโรงไฟฟ้าขยะชุมชน ขนาด 9....,<SYMBOL> TGE<ASPECT> ประมูล<OPINION> ชนะ<POS>


In [22]:
import os

save_folder = '/content/drive/MyDrive/ABSA_Datasets'
save_filename = "Text-to-Label-Dataset.csv"
dataset_pre.to_csv(os.path.join(save_folder, save_filename), index=False)